In [ ]:
import os
import numpy as np
import pandas as pd
import pickle
import sklearn.linear_model
from sklearn.model_selection import GridSearchCV
import sklearn.pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
import sklearn.metrics


# Import our custom functions
from load_data import load_data 

In [ ]:
x_tr, y_tr_df = load_data('x_train.csv', 'y_train.csv')
x_va, y_va_df = load_data('x_valid.csv', 'y_valid.csv')
x_te = load_data('x_test.csv', 'y_valid.csv')[0]


for label, arr in [('train', x_tr), ('valid', x_va)]:
    print("Contents of %s_x.csv: arr of shape %s" % (
        label, str(arr.shape)))

# Model with Duplicate Data

In [ ]:
#getting index for each class in training set:
top_int = y_tr_df.index[y_tr_df['class_name']=='top']    #index 1
trous_int = y_tr_df.index[y_tr_df['class_name']=='trouser']    #index 11
dress_int =(y_tr_df.index[y_tr_df['class_name']=='dress']).tolist()
pull_int =(y_tr_df.index[y_tr_df['class_name']=='pullover']).tolist()
sneaker_int =(y_tr_df.index[y_tr_df['class_name']=='sneaker']).tolist()
sandal_int =(y_tr_df.index[y_tr_df['class_name']=='sandal']).tolist()


#sneakers + sandals + tops + trous + dress + pullovers in new dataset.
train_x_2 = np.vstack((x_tr[sandal_int, :], x_tr[sneaker_int, :]))
tops800= np.tile(x_tr[top_int, :], (800, 1)) #make 800 copies of tops
train_x_2 = np.vstack((train_x_2, tops800)) # add 800 tops to sandals + sneakers
assert(train_x_2.shape == (2400, 784))  #checking dimensions

trous800 = np.tile(x_tr[trous_int, :], (800, 1))  #800 copies of trousers
train_x_2 = np.vstack((train_x_2, trous800)) # add 800 trousers
assert(train_x_2.shape == (3200, 784))  #checking dimensions

dress800 = np.tile(x_tr[dress_int, :], (2, 1))   #doubled dresses 400 to 800
assert(dress800.shape == (800, 784))
train_x_2 = np.vstack((train_x_2, dress800)) # add 800 dresses
assert(train_x_2.shape == (4000, 784))  #checking dimensions

pull800 = np.tile(x_tr[pull_int, :], (8, 1))
assert(pull800.shape == (800, 784))
train_x_2 = np.vstack((train_x_2, pull800)) # add 800 dresses
assert(train_x_2.shape == (4800, 784))

#making train_y_2
train_y_2 = np.tile(7, 800) 
train_y_2 = np.hstack((train_y_2, np.tile(5, 800)))
train_y_2 = np.hstack((train_y_2, np.tile(0, 800)))
train_y_2 = np.hstack((train_y_2, np.tile(1, 800)))
train_y_2 = np.hstack((train_y_2, np.tile(3, 800)))
train_y_2 = np.hstack((train_y_2, np.tile(2, 800)))

assert(train_y_2.shape == (4800, ))


In [ ]:
# Combine training and validation datasets
x_dup_all = np.vstack((train_x_2, x_va))
y_dup_all = np.hstack((train_y_2, y_va_df['class_uid']))

print("Training X shape: %s\nValidation X shape: %s\nCombined X shape: %s\n" % (train_x_2.shape, x_va.shape, x_dup_all.shape))
print("Training Y shape: %s\nValidation Y shape: %s\nCombined Y shape: %s\n" % (train_y_2.shape, y_va_df['class_uid'].shape, y_dup_all.shape))

valid_dup_indicators = np.hstack([
    -1 * np.ones(train_y_2.shape[0]), #-1 = exclude this example in test split
    0 * np.ones(y_va_df.shape[0]), #0 = include in the first test split
    ])

# Define custom splitter to use only the validation dataset for hyperparameter selection
print("Splitter dimensions: %i" % (valid_dup_indicators.shape[0]))
my_dup_splitter = sklearn.model_selection.PredefinedSplit(valid_dup_indicators)

## Random Search

In [ ]:
flag1 = 'stop'

if flag1 != 'stop':
    #batch size only for non-lbfgs. 
    #Learning Rate = const, adaptive, etc... ONLY for sgd
    #early_stopping only for sgd/adam.
    ran_params = dict(alpha = np.logspace(-5, 5, 100), learning_rate_init=np.logspace(-5, 5, 100), max_iter = [50, 100, 200, 300], hidden_layer_sizes=[(10,), (20,),(50,),(100,),(200,),(500,)])

    ran_dup_fashion_pipes = sklearn.pipeline.Pipeline([
        ('rand_search', RandomizedSearchCV(sklearn.neural_network.MLPClassifier (activation='identity', solver='lbfgs', shuffle=True, random_state=0), ran_params, scoring='balanced_accuracy', error_score='raise', return_train_score=True, n_iter=100, cv= my_dup_splitter, n_jobs = -1, refit= False, random_state=0))
    ])

    ran_dup_fashion_pipes.fit(x_dup_all, y_dup_all)
    filename = '1D_initial_RanSearch.sav'
    pickle.dump(ran_dup_fashion_pipes, open(filename, 'wb'))

In [ ]:
best_index= np.argsort(ran_dup_fashion_pipes['rand_search'].cv_results_['rank_test_score'])
for i in best_index:
    score =ran_dup_fashion_pipes['rand_search'].cv_results_['mean_test_score'][i]
    params = ran_dup_fashion_pipes['rand_search'].cv_results_['params'][i]
    print("mean_test_score: %f" % (score), "params: %s" %params)
# the BEST score 0.565 mean test

In [ ]:
flag1 = 'go'

if flag1 != 'stop':
    ran_params2 = dict(alpha = np.logspace(-5, 3, 100), learning_rate_init=np.logspace(-5, 5, 100), max_iter = [5, 10, 25, 26, 40, 290, 300, 310, 311, 312])

    ran_dup_fashion_pipes2 = sklearn.pipeline.Pipeline([
        ('rand_search', RandomizedSearchCV(sklearn.neural_network.MLPClassifier (activation='identity', solver='lbfgs', shuffle=True, random_state=0, hidden_layer_sizes= (10,)), ran_params2, scoring='balanced_accuracy', error_score='raise', return_train_score=True, n_iter=100, cv= my_dup_splitter, n_jobs = -1, refit= False, random_state=0))
    ])

    ran_dup_fashion_pipes2.fit(x_dup_all, y_dup_all)
    filename = '1D_2nd_RanSearch.sav'
    pickle.dump(ran_dup_fashion_pipes2, open(filename, 'wb'))

In [ ]:
best_index= np.argsort(ran_dup_fashion_pipes2['rand_search'].cv_results_['rank_test_score'])
for i in best_index:
    score =ran_dup_fashion_pipes2['rand_search'].cv_results_['mean_test_score'][i]
    params = ran_dup_fashion_pipes2['rand_search'].cv_results_['params'][i]
    print("mean_test_score: %f" % (score), "params: %s" %params)
# the BEST score 0.565 mean test
#mean_test_score: 0.621667 params: {'max_iter': 40, 'learning_rate_init': 2.0092330025650458e-05, 'hidden_layer_sizes': (10,), 'alpha': 62802.914418342465}

## Grid Search

In [ ]:
distEvalHype = dict(learning_rate_init = np.logspace(-2, 2, 50), alpha = np.logspace(-5, 5, 50), random_state = [0, 1, 2], max_iter = np.linspace(1, 400, 50, dtype=int))

grid_dup_fashion_pipes = sklearn.pipeline.Pipeline([
('grid_search', GridSearchCV(sklearn.neural_network.MLPClassifier
                             (activation='identity', solver='lbfgs', learning_rate= 'constant', shuffle=True, early_stopping = True, random_state=0, hidden_layer_sizes = (10, )), distEvalHype,
                              scoring='balanced_accuracy', error_score='raise',
                              return_train_score=True, cv= my_dup_splitter, n_jobs = -1, refit= False))
])
grid_dup_fashion_pipes.fit(x_dup_all, y_dup_all)


# filename = '1D_second_RanSearch.sav'
# pickle.dump(dup_fashion_pipes, open(filename, 'wb'))

In [ ]:
grid_dup_fashion_pipes['grid_search'].cv_results_

In [ ]:
results = list()
total = len(grid_dup_fashion_pipes['grid_search'].cv_results_['rank_test_score'])
for i in range(total):
    if grid_dup_fashion_pipes['grid_search'].cv_results_['rank_test_score'][i] == 1:
        results.append(grid_dup_fashion_pipes['grid_search'].cv_results_['params'][i])

print(results)